# Questionário 3

### Leitura de Dados

Hugo de Oliveira Borges  nUSP: $11915202$

In [385]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [386]:
random.seed(42) # define the seed (important to reproduce the results)

data = pd.read_csv('../data/vertebralcolumn-3C.csv', header=(0))

data = data.dropna(axis='rows') #remove NaN

# armazena os nomes das classes
nrow, ncol = data.shape
print("Matriz de atributos: Número de linhas:", nrow, " colunas: ", ncol)
attributes = list(data.columns)

print(data['class'].unique())

data.head(10)

Matriz de atributos: Número de linhas: 310  colunas:  7
['Hernia' 'Spondylolisthesis' 'Normal']


,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,Hernia
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Hernia
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Hernia
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Hernia
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Hernia
5,40.250200,13.921907,25.124950,26.328293,130.327871,2.230652,Hernia
6,53.432928,15.864336,37.165934,37.568592,120.567523,5.988551,Hernia
7,45.366754,10.755611,29.038349,34.611142,117.270067,-10.675871,Hernia
8,43.790190,13.533753,42.690814,30.256437,125.002893,13.289018,Hernia
9,36.686353,5.010884,41.948751,31.675469,84.241415,0.664437,Hernia


Como queremos escolher o melhor classificador, iremos converter para a Numpy

In [387]:
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

In [388]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

print('Media: ', np.mean(X, axis=0))
print('Desvio Padrão: ', np.std(X, axis=0))

Media:  [ 1.03143300e-16 -4.58414668e-17 -2.97969534e-16 -6.87622002e-17
  3.49541185e-16 -6.87622002e-17]
Desvio Padrão:  [1. 1. 1. 1. 1. 1.]


In [389]:
# Selecionar os conjuntos de treinamento e teste
from sklearn.model_selection import train_test_split
p = 0.8 # fracao de elementos no conjunto de treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

# definindo as classes
classes = np.unique(y_train)


### 1 - Considerando a base de dados vertebralcolumn-2C.csv, encontre o melhor classificador dentre os métodos classificador Bayesiano, classificador Bayesino não paramétrico e Naive Bayes. 

### O primeiro caso será para o classificador **Paramétrico**

Como o conjunto de treinamento e de teste estão separados, será calculado a média e o desvio padrão dos atributos de classe e em seguida teremos a classificação dos dados.

In [390]:
from scipy.stats import multivariate_normal

#matrix to store the probabilities
P = pd.DataFrame(data=np.zeros((X_test.shape[0], len(classes))), columns = classes) 

Pc = np.zeros(len(classes)) #fraction of elements in each class


for i in np.arange(0, len(classes)):
    elements = np.array(np.where(y_train == classes[i]))
    Pc[i] = elements.shape[1]/len(y_train) #Calcular a chance de pertencer
    Z = X_train[elements,:][0].astype(float)
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z))
    
    for j in np.arange(0,X_test.shape[0]):
        x = X_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P.iloc[j, i] = pj * Pc[i]
#print(P)


In [391]:
y_pred = []
#np.array(test_x.shape[0], dtype=str)
for i in np.arange(0, X_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]])) #Coluna que apresenta a maior probabilidade
    y_pred.append(classes[c])
    
y_pred = np.array(y_pred)
y_test = np.array(y_test)
#print(y_pred)

In [392]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.7903225806451613


### Agora para o caso **não paramétrico**



In [393]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from sklearn.metrics import accuracy_score

random.seed(42)

data = pd.read_csv('../data/vertebralcolumn-3C.csv', header=(0))

classes = np.array(pd.unique(data[data.columns[-1]]), dtype=str)  

# Converte para matriz e vetor do numpy
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

print('Media: ', np.mean(X, axis=0))
print('Desvio Padrão: ', np.std(X, axis=0))

Media:  [ 1.03143300e-16 -4.58414668e-17 -2.97969534e-16 -6.87622002e-17
  3.49541185e-16 -6.87622002e-17]
Desvio Padrão:  [1. 1. 1. 1. 1. 1.]


In [394]:
# Seleciona os conjuntos de treinamento e teste
p = 0.8 # fraction of elements in the training set
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

# Matriz que armazena as probabilidades para cada classe
P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes) 
Pc = np.zeros(len(classes)) # Armazena a fracao de elementos em cada classe
h = 2

for i in np.arange(0, len(classes)): # Para cada classe

    elements = tuple(np.where(y_train == classes[i])) # elmentos na classe i

    Pc[i] = len(elements)/len(y_train) # Probabilidade pertencer a classe i

    Z = x_train[elements,:][0].astype(float) # Elementos no conjunto de treinamento

    kde = KernelDensity(kernel='gaussian', bandwidth=h).fit(Z) #estimativa

    for j in np.arange(0,x_test.shape[0]): # para cada observacao no conjunto de teste

        x = x_test[j,:]
        x = x.reshape((1,len(x))) #formato aceitável para a função
        
        # calcula a probabilidade pertencer a cada classe
        pj = np.exp(kde.score_samples(x)) #log, então precisa calcular o exponencial
        P.iloc[j, i] = pj * Pc[i]
        
y_pred = [] # Vetor com as classes preditas
for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred, dtype=str)

# calcula a acuracia
score = accuracy_score(y_pred, y_test)
print('Acuracia:', score)

Acuracia: 0.8387096774193549


### Por último **Naive Bayes**

In [395]:
random.seed(42) # define the seed (important to reproduce the results)

data = pd.read_csv('../data/vertebralcolumn-3C.csv', header=(0))

data = data.dropna(axis='rows') #remove NaN

# armazena os nomes das classes
nrow, ncol = data.shape
print("Matriz de atributos: Número de linhas:", nrow, " colunas: ", ncol)
attributes = list(data.columns)

#data.head(10)


Matriz de atributos: Número de linhas: 310  colunas:  7


In [396]:
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

print('Media: ', np.mean(X, axis=0))
print('Desvio Padrão: ', np.std(X, axis=0))
print(data.shape)

Media:  [ 1.03143300e-16 -4.58414668e-17 -2.97969534e-16 -6.87622002e-17
  3.49541185e-16 -6.87622002e-17]
Desvio Padrão:  [1. 1. 1. 1. 1. 1.]
(310, 7)


In [397]:
from sklearn.model_selection import train_test_split
p = 0.8 # fraction of elements in the training set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = p, random_state = 42)

Classificação para atributos com **distribuição normal**

In [398]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

model = GaussianNB()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print('Accuracy: ', model.score(x_test, y_test))

Accuracy:  0.782258064516129


Portanto, pela melhor acurácia, vemos que o melhor classificador para a base de dados é o **Naive Bayes.**

### 2 - Considerando a base de dados “winequality-red”, encontre o valor do hiperparâmetro $h$ que oferece a maior acurácia no Bayesino não-paramétrico. 

In [399]:
random.seed(42)

data = pd.read_csv('../data/winequality-red.csv', header=(0))

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


A diferença desse exercício para os outros é o formato das classes, nesse caso temos inteiros.

In [400]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from sklearn.metrics import accuracy_score

random.seed(42)

data = pd.read_csv('../data/winequality-red.csv', header=(0))
classes = np.array(pd.unique(data[data.columns[-1]]))  

# Converte para matriz e vetor do numpy
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Seleciona os conjuntos de treinamento e teste
p = 0.8
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

h_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 1.0, 1.5, 2.0]
best_h = 0
best_score = 0

for h in h_values:
    # Matriz que armazena as probabilidades para cada classe
    P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes) 
    Pc = np.zeros(len(classes))

    for i in np.arange(0, len(classes)):
        elements = np.where(y_train == classes[i])[0]
        Pc[i] = len(elements)/len(y_train)
        
        Z = x_train[elements, :]
        kde = KernelDensity(kernel='gaussian', bandwidth=h).fit(Z)

        for j in np.arange(0,x_test.shape[0]):
            x = x_test[j,:]
            x = x.reshape((1,len(x)))
            
            pj = np.exp(kde.score_samples(x))
            P.iloc[j, i] = pj * Pc[i]
            
    y_pred = []
    for i in np.arange(0, x_test.shape[0]):
        c = np.argmax(np.array(P.iloc[[i]]))
        y_pred.append(classes[c])
    y_pred = np.array(y_pred)

    score = accuracy_score(y_pred, y_test)
    print(f'h = {h}: Acuracia = {score:.4f}')
    
    if score > best_score:
        best_score = score
        best_h = h

print(f'\nMelhor h: {best_h} com acurácia: {best_score:.4f}')

h = 0.1: Acuracia = 0.5437
h = 0.2: Acuracia = 0.6094
h = 0.3: Acuracia = 0.6406
h = 0.4: Acuracia = 0.6219
h = 0.5: Acuracia = 0.6438
h = 0.7: Acuracia = 0.6156
h = 1.0: Acuracia = 0.5844
h = 1.5: Acuracia = 0.5594
h = 2.0: Acuracia = 0.5500

Melhor h: 0.5 com acurácia: 0.6438


### 3 - Faça a classificação dos dados gerados artificialmente com o código abaixo. Compare os resultados para os métodos Naive Bayes, Classificador Bayesiano paramétrico e o classificador Bayesiano não-paramétrico (escolha um valor para h). 

In [401]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score

random.seed(42)

n_samples = 1000
data = datasets.make_moons(n_samples=n_samples, noise=.5, random_state=42)
X = data[0]
y = data[1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Selecionar os conjuntos de treinamento e teste
p = 0.8 # fracao de elementos no conjunto de treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

# definindo as classes
classes = np.unique(y_train)

#matrix to store the probabilities
P = pd.DataFrame(data=np.zeros((X_test.shape[0], len(classes))), columns = classes) 

Pc = np.zeros(len(classes)) #fraction of elements in each class

for i in np.arange(0, len(classes)):
    elements = np.array(np.where(y_train == classes[i]))
    Pc[i] = elements.shape[1]/len(y_train) #Calcular a chance de pertencer
    Z = X_train[elements,:][0].astype(float)
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z))
    
    for j in np.arange(0,X_test.shape[0]):
        x = X_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P.iloc[j, i] = pj * Pc[i]
#print(P)

y_pred = []
#np.array(test_x.shape[0], dtype=str)
for i in np.arange(0, X_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]])) #Coluna que apresenta a maior probabilidade
    y_pred.append(classes[c])
    
y_pred = np.array(y_pred)
y_test = np.array(y_test)

score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)


Accuracy: 0.795


In [402]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KernelDensity
from sklearn.metrics import accuracy_score

random.seed(42)

n_samples = 1000
data = datasets.make_moons(n_samples=n_samples, noise=.5, random_state=42)
X = data[0]
y = data[1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Selecionar os conjuntos de treinamento e teste
p = 0.8 # fraction of elements in the training set
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

# definindo as classes
classes = np.unique(y_train)

# Matriz que armazena as probabilidades para cada classe
P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes) 
Pc = np.zeros(len(classes)) # Armazena a fracao de elementos em cada classe
h = 0.4

for i in np.arange(0, len(classes)): # Para cada classe

    elements = np.where(y_train == classes[i])[0] # elementos na classe i

    Pc[i] = len(elements)/len(y_train) # Probabilidade pertencer a classe i

    Z = x_train[elements,:].astype(float) # Elementos no conjunto de treinamento

    kde = KernelDensity(kernel='gaussian', bandwidth=h).fit(Z) #estimativa

    for j in np.arange(0,x_test.shape[0]): # para cada observacao no conjunto de teste

        x = x_test[j,:]
        x = x.reshape((1,len(x))) #formato aceitável para a função
        
        # calcula a probabilidade pertencer a cada classe
        pj = np.exp(kde.score_samples(x)) #log, então precisa calcular o exponencial
        P.iloc[j, i] = pj * Pc[i]
        
y_pred = [] # Vetor com as classes preditas
for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred)

# calcula a acuracia
score = accuracy_score(y_pred, y_test)
print('Acuracia:', score)

Acuracia: 0.815


In [403]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

np.random.seed(42)

n_samples = 1000


data = datasets.make_moons(n_samples=n_samples, noise=.5, random_state=42)
X = data[0]
y = data[1]

scaler = StandardScaler()
X = scaler.fit_transform(X)

p = 0.2 # fraction of elements in the test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = p, random_state = 42)

model = GaussianNB()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print('Accuracy: ', model.score(x_test, y_test))

Accuracy:  0.795


A acurácia em todos os modelos são muito semelhantes, a única diferença aconteceu no Bayesiano não paramétrico, quando é feito alteração no hipterparâmetro $h$.
Isso pode ser causado tanto pela limitação do modelo quanto ao dataset ou até a simplicidade do dataset em relação aos modelos.